Importing Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
from bayesian_opt import BayesianOptimization
import numpy as np
import pandas as pd

This notebooks gives examples on how to use the BayesianOptimization class.
It has built in funcionality for the following:
1. Suggesting more than 1 candidate ($q>1$) per iteration.
2. 3 different types of GP models (Single Task GP, Mixed Single Task GP, SAASBO (Model List GP for $d_{out}>1$)).
3. Optional one-hot encoding when not using Mixed Single Task GP.
4. Multi-Objective and Single-Objective optimization.
5. Weights for prioritizing targets.
6. Ingredient minimization with weights.
7. 3 different types of acquisition functions for Single-Objective optimization (UCB, EI, PI).
8. Expected Hypervolume Improvement for Multi-Objective optimization.
9. 2 different $q$-sampling strategies (Believer Update and Monte-Carlo). 

Example usage of bayesian optimization for continuous data, where $d_\text{out}=1$. 

In [3]:
train_X = np.random.uniform(low = 0, high = 10, size = (10,4))
df = pd.DataFrame(train_X, columns=['pH','c1','c2', 'c3'])
df

,pH,c1,c2,c3
0,0.928565,2.061572,7.397926,1.067213
1,3.632692,0.813663,2.535886,1.946374
2,7.086042,1.906074,3.630353,5.756157
3,9.576056,7.908410,5.428949,0.189259
4,0.177517,2.204509,4.778828,8.870002
5,6.318075,2.550826,2.290025,1.753717
6,0.340457,6.616294,3.050574,6.763635
7,2.306825,8.295865,0.844516,3.342043
8,4.475850,0.238764,9.066787,3.608162
9,2.666601,6.106191,1.323932,6.682797


In [4]:
bo_model = BayesianOptimization().fit(df, ['c3'], model_type='Single-Task GP')

a = bo_model.candidates(1, export_df=True)
a


,pH,c1,c2,c3
0,2.97,6.14,0.99,5.180145


Example usage of bayesian optimization for mixed continuous and categorical data with target of dim 1 

In [5]:
train_X_cat = train_X
train_X_cat[:,0] = np.round(train_X_cat[:,0] )
df_cat = pd.DataFrame(train_X_cat, columns=['pH','c1','c2','c3'])
df_cat


,pH,c1,c2,c3
0,1.0,2.061572,7.397926,1.067213
1,4.0,0.813663,2.535886,1.946374
2,7.0,1.906074,3.630353,5.756157
3,10.0,7.908410,5.428949,0.189259
4,0.0,2.204509,4.778828,8.870002
5,6.0,2.550826,2.290025,1.753717
6,0.0,6.616294,3.050574,6.763635
7,2.0,8.295865,0.844516,3.342043
8,4.0,0.238764,9.066787,3.608162
9,3.0,6.106191,1.323932,6.682797


In [6]:
bo_model_mst = BayesianOptimization().fit(df_cat,['c3'],cat_dims=['pH'], model_type='Mixed Single-Task GP')

cand_1 = bo_model_mst.candidates(1, export_df=True)
cand_1

,pH,c1,c2,c3
0,2.0,2.2,4.78,6.162675


In [7]:
bo_model_st = BayesianOptimization().fit(df_cat,['c3'],cat_dims=['pH'], model_type='Single-Task GP')

cand_2 = bo_model_st.candidates(1, export_df=True)
cand_2

,pH,c1,c2,c3
0,0.0,2.13,4.51,7.978325


Choosing a different GP model type.

In [8]:
bo_model = BayesianOptimization().fit(df,['c3'],cat_dims=['pH'], model_type='SAASBO')

a = bo_model.candidates(2, export_df=True)
a

,pH,c1,c2,c3
0,0.0,3.32,4.44,7.988038
1,0.0,1.42,4.77,7.710234


Different $q$-sampling strategy.

In [9]:
bo_model = BayesianOptimization().fit(df,['c3'])

a = bo_model.candidates(4, q_sampling_method="Believer", export_df=True)
a

,pH,c1,c2,c3
0,7.14,2.30,4.00,4.504538
1,1.62,4.94,1.66,4.043994
2,3.18,6.14,1.73,5.235239
3,0.05,6.18,3.11,5.331708


Choosing a different acquisition function.

In [10]:
a = bo_model.candidates(4, export_df=True, acq_func_name="UCB")
a

,pH,c1,c2,c3
0,0.50,0.57,2.66,4.135625
1,4.12,6.91,1.06,4.136613
2,0.79,7.88,1.73,4.135573
3,0.09,6.58,3.06,6.649267


Minimizing Ingredients

In [11]:
a = bo_model.candidates(4, q_sampling_method="Believer", export_df=True, input_weights={0:1.1, 1:2.5, 2:2.0})
a

,pH,c1,c2,c3
0,0.00,0.24,0.84,4.135625
1,0.63,0.24,0.84,4.036131
2,0.00,0.24,1.41,4.046192
3,1.29,0.24,0.84,4.014650


Multi-Objective optimization.

In [12]:
bo_model = BayesianOptimization().fit(df,['c2', 'c3'])

a = bo_model.candidates(4, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\acquisition\multi_objective\monte_carlo.py:111: NumericsWarning: qExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qExpectedHypervolumeImprovement 	 --> 	 qLogExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


,pH,c1,c2,c3
0,5.29,6.98,4.034778,3.997936
1,0.00,2.20,4.773868,8.836711
2,4.21,2.81,4.034778,3.997936
3,0.74,2.10,6.272942,2.235691
